## Project 1.3 - Test Functions 

Group: Emmy Noether

Students: Janik Rausch (628334), Camilo Tello Breuer (633291), Ida Wöstheinreich (628428)

In [1]:
using LinearAlgebra,Random,Printf,FFTW,JLD,Plots,LaTeXStrings,Colors
include("Ground_state.jl"); include("Evolution.jl")

simulate_2D

### Two important properties
- **The unitarity of integrators**\
    The exact evolution operator is a unitary operator, i.e. it satisfies 
    $$U_\text{ex}^\dagger U_\text{ex} =1.$$
    The unitarity property in the context of quantum mechanics is also often called $\textit{conservation of total probability}$, because this property is equivalent to the statement that the norm of states is conserved by the time evolution. 

- **The energy conservation of integrators**\
    The exact evolution operator satisfies
    $$U_\text{ex}^\dagger \hat{H}U_\text{ex} =\hat{H}$$
    This property implies that the expectation value of the energy does not depend on time, i.e. it is conserved.

In [60]:
function test_unitarity(N::Int, μ::Number, ϵ::Number, τ::Number, ψ_0::Array, method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    
    n = lattice(N, 1)
    V = potential(μ, ϵ, n)
    norm_ψ_0 = norm(ψ_0)
    ψ = integrator(μ, ϵ, ψ_0, V, τ)
    norm_ψ = norm(ψ)
    residue = abs(norm_ψ - norm_ψ_0)
    print("The difference between the norm of states before and after a time step(τ = "); print(τ) 
    print(") is \n"); print(residue)
end 
function test_energy_conservation(N::Int, μ::Number, ϵ::Number, τ::Number, ψ_0::Array, method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    n = lattice(N, 1)
    V = potential(μ, ϵ, n)
    E_0,_ = energy_expectation(ψ_0, μ, ϵ)
    ψ = integrator(μ, ϵ, ψ_0, V, τ)
    E,_= energy_expectation(ψ, μ, ϵ)
    residue = abs(E - E_0)
    print("\nThe difference between the energy expectation before and after a time step(τ = "); print(τ)
    print(") is \n"); print(residue)
end
    


test_energy_conservation (generic function with 1 method)

In [61]:
N = 200
τ = 5
μ, ϵ = 20, sqrt(0.001)
m, k = (1/ϵ,), (1,)
σ = 10
ψ_0 = wave_packet(lattice(N, 1), m, σ, k)
method = ["euler", "crank-nicolson", "strang-splitting"]
test_unitarity(N, μ, ϵ, τ, ψ_0, method[3])
test_energy_conservation(N, μ, ϵ, τ, ψ_0, method[3])

The difference between the norm of states before and after a time step(τ = 5) is 
2.220446049250313e-16
The difference between the energy expectation before and after a time step(τ = 5) is 
26.858525243349877

In [115]:
function test_unitarity(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number , method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)] # length(m) == ndims(n) 
    
    function test_engine(shape::Tuple{Vararg{Int}})   
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)])
        norm_ψ_0 = norm(ψ_0)
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        norm_ψ = norm(ψ)
        residue = abs(norm_ψ - norm_ψ_0) 
        @printf "residue = %.3e" residue 
    end
    print("\nConservation of the Norm of the states with the "); print(method); print(" method after a time step(τ = "); print(τ) 
    print("): \n");
    print("D=1: "); test_engine((N,))
    if method != "strang-splitting"
        print("\nD=2: "); test_engine((N,N))
        #print("\nD=3: "); test_engine((N,N,N)); print("\n"))   
    end
end 
function test_energy_conservation(N::Int, μ::Number, ϵ::Number, τ::Number, σ::Number, method::String; tol_cg::Number=1e-10, maxiters_cg::Int=10000)
    integrator = choose_integrator(method; tol_cg, maxiters_cg)
    m, k = [(1/ϵ,),(1/ϵ,0),(1/ϵ,0, 0)], [(1,),(1,0),(1,0,0)]
    function test_engine(shape::Tuple{Vararg{Int}})  
        n = lattice(N, length(shape))
        V = potential(μ, ϵ, n)
        ψ_0 = wave_packet(n, m[length(shape)], σ, k[length(shape)])
        E_0,_ = energy_expectation(ψ_0, μ, ϵ)
        ψ = integrator(μ, ϵ, ψ_0, V, τ)
        E,_= energy_expectation(ψ, μ, ϵ)
        residue = abs(E - E_0)
        @printf "residue = %.3e" residue
    end
    print("\nConservation of the energy expectation with the "); print(method); print(" method after a time step(τ = "); print(τ)
    print("): \n"); 
    print("D=1: "); test_engine((N,))
    if method != "strang-splitting"
        print("\nD=2: "); test_engine((N,N))
        #print("\nD=3: "); test_engine((N,N,N)); print("\n"))   
    end
    
end

ErrorException: syntax: break or continue outside loop

Not all approximated integrators preserve unitarity and energy conservation exactly.
For example, the `Euler integrator` is neither unitary nor energy-preserving.  
But both properties are recovered in the τ → 0 limit:

In [110]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[1])
    test_energy_conservation(N, μ, ϵ, t, σ, method[1])
end


Conservation of the Norm of the states with the euler method after a time step(τ = 0.5): 
D=1: residue = 1.093e+01
D=2: residue = 1.101e+01
Conservation of the energy expectation with the 

euler method after a time step(τ = 0.5): 
D=1: residue = 3.440e+03
D=2: residue = 3.509e+03
Conservation of the Norm of the states with the euler method after a time step(τ = 0.05): 
D=1: residue = 5.532e-01
D=2: residue = 5.593e-01
Conservation of the energy expectation with the euler method after a time step(τ = 0.05): 
D=1: residue = 3.440e+01
D=2: residue = 3.509e+01
Conservation of the Norm of the states with the euler method after a time step(τ = 0.0005): 
D=1: residue = 7.063e-05
D=2: 

residue = 7.157e-05
Conservation of the energy expectation with the euler method after a time step(τ = 0.0005): 
D=1: residue = 3.440e-03
D=2: residue = 3.509e-03

But the `Crank-Nicolson` integrator is unitary and conserves the energy. 
The residues for these method are consistently of order $10^{-10}$ or smaller for both tests.
The size of the residues does not depend on the size of τ:

In [111]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[2])
    test_energy_conservation(N, μ, ϵ, t, σ, method[2])
end


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.5): 
D=1: 

residue = 2.201e-12
D=2: 

residue = 1.725e-13
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.5): 
D=1: residue = 4.946e-10


D=2: residue = 1.478e-12


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 2.340e-13
D=2: 

residue = 3.273e-13
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.05): 
D=1: residue = 1.022e-10
D=2: residue = 8.825e-12


Conservation of the Norm of the states with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 2.887e-15
D=2: 

residue = 5.573e-14
Conservation of the energy expectation with the crank-nicolson method after a time step(τ = 0.0005): 
D=1: residue = 2.441e-12
D=2: residue = 1.720e-12

The `Strang-splitting` integrator is unitary and but does not conserve the energy. 
The residue for the unitary test are consistently of order $10^{-10}$ or smaller and
their size does not depend on the size of τ. Energy conservation is recovered in the τ → 0 limit:

In [113]:
N = 200
τ = [0.5, 0.05, 0.0005]
μ, ϵ = 20, sqrt(0.001)
σ = 10
method = ["euler", "crank-nicolson", "strang-splitting"]
for t in τ
    test_unitarity(N, μ, ϵ, t, σ, method[3])
    test_energy_conservation(N, μ, ϵ, t, σ, method[3])
end


Conservation of the Norm of the states with the strang-splitting method after a time step(τ = 0.5): 
D=1: 

ArgumentError: ArgumentError: FFTW plan applied to wrong-size array



The Crank-Nicolson is the only one that conserves energy while the other two integrators do not. 
The residue for these method is consistently of order $10^{-6}$ or smaller for the energy conservation test.
Obviously both properties are recovered for all integrators in the τ → 0 limit.